In [ ]:
# set up our local code path
import sys
sys.path.append('/home/serenity/dev/shadows/serenity/serenity-mdrecorder/src')

# standard imports
import pandas as pd
import plotly.graph_objects as go

from cloudwall.serenity.mdrecorder.tickstore import LocalTickstore
from datetime import datetime as dt
from pathlib import Path
from plotly.subplots import make_subplots

# set up Cufflinks
import cufflinks as cf
cf.set_config_file(theme='ggplot',sharing='public',offline=True)

In [ ]:
# Connect to Behemoth
phemex_tickstore = LocalTickstore(Path('/behemoth/db/PHEMEX_TRADES'), 'time')


In [ ]:
phemex_trades = phemex_tickstore.select('BTCUSD', start=dt(2020, 4, 1), end=dt(2020, 12, 31))

In [ ]:
phemex_trades['price'] = phemex_trades['price'] / 10000
phemex_trades.head()

In [ ]:
cbp_tickstore = LocalTickstore(Path('/behemoth/db/COINBASE_PRO_TRADES'), 'time')
cbp_trades = cbp_tickstore.select('BTC-USD', start=dt(2020, 4, 1), end=dt(2020, 12, 31))

In [ ]:
binance_tickstore = LocalTickstore(Path('/behemoth/db/BINANCE_TRADES'), 'time')
binance_trades = binance_tickstore.select('BTC-USDT', start=dt(2020, 4, 1), end=dt(2020, 12, 31))

In [ ]:
cbp_5m = cbp_trades['price'].resample('5Min').ohlc()

In [ ]:
binance_5m = binance_trades['price'].resample('5Min').ohlc()

In [ ]:
phemex_5m = phemex_trades['price'].resample('5Min').ohlc()

In [ ]:
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(x=phemex_5m.index, y=phemex_5m['close'], name="Phemex")
)


fig.add_trace(
    go.Scatter(x=cbp_5m.index, y=cbp_5m['close'], name="Coinbase Pro")
)

fig.add_trace(
    go.Scatter(x=binance_5m.index, y=binance_5m['close'], name="Binance")
)

# Add figure title
fig.update_layout(
    title_text="Phemex futures vs. Coinbase spot & USDT (5m)"
)

fig.show()

In [ ]:
join_5m = phemex_5m.join(cbp_5m, on='time', how='left', lsuffix="_phemex", rsuffix="_cbp")
join_5m = join_5m.join(binance_5m, on='time', how='left')
join_5m

In [ ]:
# compute the basis vs. BTC-USD and BTC-USDT (Tether) spot prices
join_5m['basis_btcusd'] = join_5m['close_cbp'] - join_5m['close_phemex']
join_5m['basis_btcusdt'] = join_5m['close'] - join_5m['close_phemex']

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=join_5m.index, y=join_5m['close_phemex'], name="Phemex")
)

fig.add_trace(
    go.Scatter(x=join_5m.index, y=join_5m['basis_btcusd'], name="Basis (BTC-USD)"), secondary_y=True
)

fig.add_trace(
    go.Scatter(x=join_5m.index, y=join_5m['basis_btcusdt'], name="Basis (BTC-USDT)"), secondary_y=True
)

# Add figure title
fig.update_layout(
    title_text="Phemex futures vs. basis (5m)"
)

fig.show()

In [ ]:
phemex_volume_5m = phemex_trades.resample('5Min').agg({'size': 'sum'})

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=phemex_5m.index, y=join_5m['close'], name="Price")
)

fig.add_trace(
    go.Bar(x=phemex_volume_5m.index, y=phemex_volume_5m['size'], name="Volume"), secondary_y=True
)

# Add figure title
fig.update_layout(
    title_text="Phemex futures vs. volumes (5m)"
)

fig.show()